In [49]:
import numpy as np
import xarray as xr
import rasterio
%matplotlib inline
from matplotlib.pyplot import *
from glob import glob
import os
import datetime

from rasterio_to_xarray import rasterio_to_xarray, xarray_to_rasterio

In [50]:
def maiac_file_to_da(filename):
    da = rasterio_to_xarray(filename)
    
    da.values[da.values == -28672] = np.nan
    da.values[da.values == 0] = np.nan
    
    #da.values = da.values.astype(np.float64)
    
    time_str = os.path.basename(filename)[17:-13]
    time_obj = datetime.datetime.strptime(time_str, '%Y%j%H%M')
    da.coords['time'] = time_obj
    
    return da

In [51]:
files = glob('2014_All/*_proj.tif')

In [52]:
list_of_das = map(maiac_file_to_da, files)

In [53]:
res = xr.concat(list_of_das, 'time')

/Users/robin/code/MAIACProcessing/rasterio_to_xarray.py:29: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  data = np.where(data == src.nodata, np.nan, data)


In [54]:
r = res.resample('D', dim='time', how='max')

In [55]:
ds = r.to_dataset(name='data')

In [57]:
ds.to_netcdf('robin_2014_All.nc')

In [58]:
all_data = xr.open_mfdataset('robin_2014_All.nc', chunks={'time': 10})

In [59]:
all_data = all_data['data']

In [60]:
all_data.chunks

((10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1),
 (1162,),
 (1240,))

In [61]:
xarray_to_rasterio(all_data.mean(dim='time', keep_attrs=True), 'test_dask_overallmean2.tif')

/Users/robin/anaconda3/lib/python3.5/site-packages/dask/array/numpy_compat.py:44: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
